# Word Embeddings for Sentiment Analysis

This notebook explains how to add the embeddings layer to the neural network. We will train our own word embeddings using a simple Keras model for a sentiment classification task.

Steps include:
1. Downloading data from tensorflow dataset.
2. Segregating training and testing sentences & labels.
3. Data preparation to padded sequences
4. Defining out Keras model with an Embedding layer.
5. Train the model and explore the weights from the embedding layer.


In [64]:
##import the required libraries and APIs
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

print(tf.__version__)

2.14.0


## Downloading the TensorFlow `imdb_review` dataset

> Make sure tensorflow_datasets is installed

In [65]:
##load the imdb reviews dataset
data, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)

## Segregating training and testing sets

In [66]:
##segregate training and test set
train_data, test_data = data['train'], data['test']

##create empty list to store sentences and labels
train_sentences = []
test_sentences = []

train_labels = []
test_labels = []

In [67]:
##iterate over the train data to extract sentences and labels
for sent, label in train_data:
    train_sentences.append(str(sent.numpy().decode('utf8')))
    train_labels.append(label.numpy())

##iterate over the test set to extract sentences and labels
for sent, label in test_data:
    test_sentences.append(str(sent.numpy().decode('utf8')))
    test_labels.append(label.numpy())



In [68]:
##convert lists into numpy array
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

## Data preparation - setting up the tokenizer

In [69]:
##define the parameters for the tokenizing and padding
vocab_size = 10000
embedding_dim = 16
max_length = 150
trunc_type='post'
oov_tok = "<oov>"


In [70]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

##training sequences and labels
train_seqs = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_seqs, maxlen=max_length, truncating=trunc_type)

##testing sequences and labels
test_seqs = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_seqs,maxlen=max_length, truncating=trunc_type)

In [71]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(train_sentences[1])
print(train_padded[1])
print(decode_review(train_padded[1]))

I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot development was constant. Constantly slow and boring. Things seemed to happen, but with no explanation of what was causing them or why. I admit, I may have missed part of the film, but i watched the majority of it and everything just seemed to happen of its own accord without any real concern for anything else. I cant recommend this film at all.
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0   11   26   75  571
    6  805 2354  313  106   19   12    7  629  686    6    4 2219    5
  181  584   64 1454  110 2263    3 3951   21    2    1    3  258   41
 4677    4  174  188   21  

## Define the Neural Network with Embedding layer

1. Use the Sequential API.
2. Add an embedding input layer of input size equal to vocabulary size.
3. Add a flatten layer, and two dense layers.

In [74]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAvgPool1D(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

##compile the model with loss function, optimizer and metrics
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 150, 16)           160000    
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense_6 (Dense)             (None, 6)                 102       
                                                                 
 dense_7 (Dense)             (None, 1)                 7         
                                                                 
Total params: 160109 (625.43 KB)
Trainable params: 160109 (625.43 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## Model Training

In [75]:
num_epochs = 10

##train the model with training and validation set
model.fit(
    train_padded,
    train_labels,
    epochs=num_epochs,
    validation_data=(test_padded, test_labels)
    )

Epoch 1/10
782/782 [==============================] - 29s 36ms/step - loss: 0.5680 - accuracy: 0.7574 - val_loss: 0.4245 - val_accuracy: 0.8247
Epoch 2/10
782/782 [==============================] - 10s 13ms/step - loss: 0.3363 - accuracy: 0.8676 - val_loss: 0.3536 - val_accuracy: 0.8462
Epoch 3/10
782/782 [==============================] - 9s 12ms/step - loss: 0.2693 - accuracy: 0.8950 - val_loss: 0.3419 - val_accuracy: 0.8520
Epoch 4/10
782/782 [==============================] - 6s 8ms/step - loss: 0.2326 - accuracy: 0.9109 - val_loss: 0.3481 - val_accuracy: 0.8496
Epoch 5/10
782/782 [==============================] - 6s 8ms/step - loss: 0.2065 - accuracy: 0.9223 - val_loss: 0.3699 - val_accuracy: 0.8421
Epoch 6/10
782/782 [==============================] - 5s 7ms/step - loss: 0.1856 - accuracy: 0.9331 - val_loss: 0.3977 - val_accuracy: 0.8362
Epoch 7/10
782/782 [==============================] - 6s 8ms/step - loss: 0.1681 - accuracy: 0.9401 - val_loss: 0.4121 - val_accuracy: 0.8365
E

## Deriving weights from the embedding layer

In [78]:
##isolating the first embedding layer
l1 = model.layers[0]

##extracting learned weights
weights = l1.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)
print(weights)

(10000, 16)
[[ 0.16401646 -0.01223065 -0.04520727 ... -0.04703828 -0.01448146
   0.0517274 ]
 [ 0.18435945  0.03284742  0.00592576 ... -0.08042753 -0.0223888
   0.07725939]
 [ 0.32169956 -0.02697626  0.02371125 ... -0.07672437 -0.01251747
   0.08873405]
 ...
 [ 0.22980613 -0.17263031  0.14889506 ...  0.23769124  0.23568067
  -0.20487857]
 [ 0.31796062 -0.4089856   0.40573272 ...  0.39037916  0.3905874
  -0.31358936]
 [ 0.22681564 -0.34258097  0.27681047 ...  0.30871683  0.24986698
  -0.2798837 ]]
